In [1]:
import os

In [2]:
%pwd

'd:\\AI\\Kidney-Disease-Classification-MLflow-DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\AI\\Kidney-Disease-Classification-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfiq:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    parms_epoch: int
    parms_batch_size: int
    parms_is_augumentaion: bool
    parms_image_size: list

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.comman import read_yaml,create_path

In [7]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,parms_path = PARMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.parms = read_yaml(parms_path)
        create_path([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfiq:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        parms = self.parms
        trainig_data = os.path.join(self.config.data_ingestion.unzip_dir,'Kidney-CT-Scan')

        create_path([Path(training.root_dir)])

        training_confiq=TrainingConfiq(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.training_model_path),
            updated_base_model_path = Path(prepare_base_model.base_model_updated_path),
            training_data = trainig_data,
            parms_epoch = parms.EPOCHS,
            parms_batch_size = parms.BATCH_SIZE,
            parms_is_augumentaion = parms.AUGUMENTATION,
            parms_image_size = parms.IMAGE_SIZE
            )
        return training_confiq

In [8]:
import os
import tensorflow as tf

[2024-08-31 02:38:28,879: INFO: utils: NumExpr defaulting to 4 threads.]


In [9]:
class Training:
    def __init__(self,config:TrainingConfiq):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generater(self):
        datagenerater_kwargs = dict(rescale = 1./255,validation_split = 0.20)
        dataflow_kwargs= dict(target_size = self.config.parms_image_size[:-1],batch_size = self.config.parms_batch_size,interpolation = 'bilinear')
        valid_datagenerater = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerater_kwargs)

        self.valid_generator = valid_datagenerater.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.parms_is_augumentaion:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerater_kwargs
            )
        else:
            train_datagenerator = valid_datagenerater

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.parms_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generater()
    training.train()
    
except Exception as e:
    raise e

config\config.yaml
[2024-08-31 02:38:31,943: INFO: comman: yaml file:config\config.yaml load sucessfully]
params.yaml
[2024-08-31 02:38:31,959: INFO: comman: yaml file:params.yaml load sucessfully]
[2024-08-31 02:38:31,959: INFO: comman: Directorys:artifacts sucessfully]
[2024-08-31 02:38:31,959: INFO: comman: Directorys:artifacts\training sucessfully]
Found 80 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
20/20 [==============================] - 901s 45s/step - loss: 1.1649 - accuracy: 0.4938 - val_loss: 0.6927 - val_accuracy: 0.5000
